In [1]:
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '8g'), ('spark.driver.memory','15g'),("spark.driver.maxResultSize","0"),("spark.executor.cores", '2'),("spark.executor.instances", "8"),("spark.dynamicAllocation.enabled","False")])

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1649265864775_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.storagelevel import StorageLevel

from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Transformer, classification
from pyspark.ml import tuning
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df = sqlContext.read.parquet("hdfs:///NLP-parquet/").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = df.withColumnRenamed("stars","label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8

In [7]:
df = df.repartition(800)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

800

In [9]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- rawFeatures: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

In [10]:
#gbt = Classifier(featuresCol='features',labelCol="stars")
#model = DecisionTree.trainClassifier(sc.parallelize(df), 5, {})
#NB = NaiveBayes(featuresCol='features', labelCol='label', smoothing=1.0, modelType='multinomial', thresholds=None, weightCol=None)
#rf = classification.DecisionTreeClassifier(featuresCol='features',maxDepth=3, labelCol="stars",predictionCol='prediction', seed=42)

lr = classification.LogisticRegression(maxIter=100, regParam=0.03, elasticNetParam=0.1, tol=1E-6, standardization=True, family="multinomial")
#lr = classification.LogisticRegression(family="multinomial")
ovr = classification.OneVsRest(classifier=lr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
ovrModel = ovr.fit(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:

# paramGrid = tuning.ParamGridBuilder().addGrid(lr.maxIter, [30,50,75]) \
#                               .addGrid(lr.regParam, [0.03, 0.04, 0.05]) \
#                               .addGrid(lr.elasticNetParam, [0.1,0.2,0.3]) \
#                               .build()
# evaluator = MulticlassClassificationEvaluator()
# valid = tuning.TrainValidationSplit(estimator=lr,
#                              estimatorParamMaps=paramGrid,
#                              evaluator=evaluator,
#                              trainRatio=0.80)
# model = valid.fit(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-12:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1397



In [14]:
best_model = model.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#rf_model = rf.fit(df)

#lrModel = ovr.fit(df)

In [16]:
result = ovrModel.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#best_model.printSchema()
result.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- rawFeatures: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = true)

In [ ]:
#best_model.show()
result.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ovrModel.explainParam()

In [25]:
#best_model = model.bestModel.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='LogisticRegression_664b4ff26ea8', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2, Param(parent='LogisticRegression_664b4ff26ea8', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.1, Param(parent='LogisticRegression_664b4ff26ea8', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'multinomial', Param(parent='LogisticRegression_664b4ff26ea8', name='featuresCol', doc='features column name'): 'features', Param(parent='LogisticRegression_664b4ff26ea8', name='fitIntercept', doc='whether to fit an intercept term'): True, Param(parent='LogisticRegression_664b4ff26ea8', name='labelCol', doc='label column name'): 'label', Param(parent='LogisticRegression_664b4ff26ea8', name='maxIter', doc='maximum number of iterations (>= 0)'):

In [26]:
best_model_t = model.bestModel.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(result,{evaluator.metricName: "accuracy"})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5425743578759885

In [7]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…